<a href="https://colab.research.google.com/github/Vastoud/OPTI/blob/main/Taller_optimizacion_Gonzalo_Rivera_y_Sebastian_Riquelme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pulp

from pulp import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 63.3 MB/s eta 0:00:00


In [2]:
# Datos del problema
trabajadores = ["Trabajador 1", "Trabajador 2", "Trabajador 3", "Trabajador 4", "Trabajador 5"]
tareas = ["Tarea 1", "Tarea 2", "Tarea 3", "Tarea 4", "Tarea 5", "Tarea 6", "Tarea 7", "Tarea 8", "Tarea 9", "Tarea 10"]

costos = {
    ("Trabajador 1", "Tarea 1"): 20, ("Trabajador 1", "Tarea 2"): 20, ("Trabajador 1", "Tarea 6"): 20, ("Trabajador 1", "Tarea 7"): 20,
    ("Trabajador 2", "Tarea 1"): 25, ("Trabajador 2", "Tarea 3"): 25, ("Trabajador 2", "Tarea 6"): 25, ("Trabajador 2", "Tarea 8"): 25,
    ("Trabajador 3", "Tarea 1"): 22, ("Trabajador 3", "Tarea 4"): 22, ("Trabajador 3", "Tarea 6"): 22, ("Trabajador 3", "Tarea 9"): 22,
    ("Trabajador 4", "Tarea 2"): 23, ("Trabajador 4", "Tarea 5"): 23, ("Trabajador 4", "Tarea 7"): 23, ("Trabajador 4", "Tarea 10"): 23,
    ("Trabajador 5", "Tarea 3"): 21, ("Trabajador 5", "Tarea 4"): 21, ("Trabajador 5", "Tarea 8"): 21, ("Trabajador 5", "Tarea 9"): 21
}

tiempo_tareas = {
    "Tarea 1": 10, "Tarea 2": 7, "Tarea 3": 5, "Tarea 4": 6, "Tarea 5": 8, "Tarea 6": 10, "Tarea 7": 7, "Tarea 8": 5, "Tarea 9": 6, "Tarea 10": 8
}

fecha_limite_tareas = {
    "Tarea 1": 3, "Tarea 2": 2, "Tarea 3": 2, "Tarea 4": 3, "Tarea 5": 4, "Tarea 6": 5, "Tarea 7": 6, "Tarea 8": 7, "Tarea 9": 8, "Tarea 10": 9
}

costo_retraso = 50

In [3]:
prob = LpProblem("AsignacionTareas", LpMinimize)

In [4]:
#Variable Decision
asignacion = LpVariable.dicts("Asignacion", (trabajadores, tareas), cat='Binary')

In [5]:
#Función objetivo
func_objetivo = lpSum(costos[(trabajador, tarea)] * asignacion[trabajador][tarea]
                     for trabajador in trabajadores for tarea in tareas
                     if (trabajador, tarea) in costos)

multas_retraso = lpSum(costo_retraso * lpSum((fecha_limite_tareas[tarea] - dia) * asignacion[trabajador][tarea]
                                             for dia in range(fecha_limite_tareas[tarea] + 1, 100))
                       for trabajador in trabajadores for tarea in tareas
                       if (trabajador, tarea) in fecha_limite_tareas)

prob += func_objetivo + multas_retraso





In [6]:
#Restricciones

for tarea in tareas:
    prob += lpSum(asignacion[trabajador][tarea] for trabajador in trabajadores) == 1

for trabajador in trabajadores:
    prob += lpSum(asignacion[trabajador][tarea] for tarea in tareas) <= 1


In [7]:
prob.solve()
print("Estado:", LpStatus[prob.status])

Estado: Infeasible


In [8]:
print("Costo total:", value(prob.objective))
print("\nAsignación de tareas:")
for trabajador in trabajadores:
    for tarea in tareas:
        if asignacion[trabajador][tarea].value() == 1:
            print(f"{trabajador} - {tarea}")

Costo total: 0.0

Asignación de tareas:
Trabajador 1 - Tarea 9
Trabajador 2 - Tarea 2
Trabajador 2 - Tarea 4
Trabajador 2 - Tarea 9
Trabajador 3 - Tarea 3
Trabajador 3 - Tarea 7
Trabajador 3 - Tarea 10
Trabajador 4 - Tarea 1
Trabajador 4 - Tarea 8
Trabajador 5 - Tarea 6
